<center>
<table>
  <tr>
    <td><img src="http://www.nasa.gov/sites/all/themes/custom/nasatwo/images/nasa-logo.svg" width="100"/> </td>
     <td><img src="https://github.com/astg606/py_materials/blob/master/logos/ASTG_logo.png?raw=true" width="80"/> </td>
     <td> <img src="https://www.nccs.nasa.gov/sites/default/files/NCCS_Logo_0.png" width="130"/> </td>
    </tr>
</table>
</center>

        
<center>
<h1><font color= "blue" size="+3">ASTG Python Courses</font></h1>
</center>

---

<center><h1> <font color="red">Reading LANDSAT hdf Files using pyhdf</font></h1></center>

## <font color="red">Primary References/Resources</font>

- https://moonbooks.org/Articles/How-to-read-a-MODIS-HDF-file-using-python-/
- http://fhs.github.io/pyhdf/modules/SD.html 

### Import Statements:

In [ ]:
import pprint
import os

In [ ]:
import numpy as np
import xarray as xr

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import hvplot.xarray
from cartopy import crs as ccrs

In [ ]:
from pyhdf.SD import SD
from pyhdf.SD import SDS
from pyhdf.SD import SDC
from pyhdf.SD import SDim
from pyhdf.SD import SDAttr

In [ ]:
# Toggles off alphabetical sorting
pprint.sorted = lambda x, key=None:x

## <font color="red">LANDSAT File Name Convention</font>

Details on the naming convention can be found in:

https://gisgeography.com/landsat-file-naming-convention/
    


## <font color="red"> Accessing a Sample HDF4 Data Files</font>

In [ ]:
data_dir = "/Users/jkouatch/myTasks/PythonTraining/ASTG606/Materials/sat_data/LANDSAT_Data"

In [ ]:
file_name = os.path.join(data_dir, "LT50830152011198GLC00.hdf")

### <font color="blue"> Opening a File</font>

In [ ]:
# Opening file for reading
fid = SD(file_name, SDC.READ)

Some file information only work if attributes exist:

In [ ]:
# print("Author:", fid.author)
# print("Priority:", fid.priority)

Basic information on the files:

- The first number indicates the number of datasets in the file (not to be confused w/ xarray datasets)
- The second number indicates the number of attributes attached to the global file.

In [ ]:
fid.info()

#### File Attributes

- We can access the file attributes which hold important global metadata.
- Some notable ones are the data provider, satellite name and instrument, coordinate boundaries, and structural metadata.

In [ ]:
file_attrs = fid.attributes()
pprint.pprint(file_attrs)

We can also access the datasets' names and basic info such as shape and dimension labels

In [ ]:
file_dts = fid.datasets()
pprint.pprint(file_dts)

In [ ]:
for index, name in enumerate(file_dts.keys(), start=1):
    print(index, name)

### <font color="blue">Data Extraction as NumPy Arrays</font>

Let's assume that we want to extract data from the field `sr_band4`.

The `select()` method from the `SD` class allows us to extract a dataset (object) given it's name or index number.

In [ ]:
sample_ds = fid.select('sr_band4') 

Get basic information from the dataset:

- The `info()` function in the `SDS` class allows us to get the dataset name, rank (or level with file-leve being rank 1), dimension lengths, data type, and number of attributes.

In [ ]:
sample_ds.info()

List the dataset attributes:

In [ ]:
ds_attrs = sample_ds.attributes()
ds_attrs

Important information such as `fill values`, `scale factors`, and `offset values` are found in dataset attributes and will be important to fully restoring the data.

In [ ]:
# Default values (also a reset if testing different datasets/variables)
fill = None
scale = 1
offset = 0

In [ ]:
for key, value in sample_ds.attributes().items():
    if key == '_FillValue':
        fill = value  
    if key == 'scale_factor':
        scale = value
    if key == 'add_offset':
        offset = value
# data = data * scale + offset
    
print('Fill Value:', fill)
print('Scale Factor:', scale)
print('Offset:', offset)

#### Extract the data

- We can retrieve and store the data itself as a NumPy array using the `get()` function.

In [ ]:
sample_data = sample_ds.get() 

Confirms that the data has been stored as a NumPy array.

In [ ]:
print("Dataset Class Type: ", type(sample_data))

In [ ]:
sample_data

Just like any NumPy array, we can get the shape and dtype.

In [ ]:
sample_data.shape

In [ ]:
sample_data.dtype

#### DATA TYPES

0 - SDC.UNLIMITED: dimensions only; they can grow dynamically

3 - SDC.UCHAR; SDC.UCHAR8: unsigned 8-bit integer

4 - SDC.CHAR; SDC.CHAR8: 8-bit character

5 - SDC.FLOAT32: 32-bit floating point

6 - SDC.FLOAT64: 64-bit floating point

20 - SDC.INT8: signed 8-bit integer

21 - SDC.UINT8: unsigned 8-bit integer

22 - SDC.INT16: signed 16-bit integer

23 - SDC.UINT16: unsigned 16-bit integer

24 - SDC.INT32: signed 32-bit integer

25 - SDC.UINT32: unsigned 32-bit integer

### <font color="blue">Attributes</font>

There are three levels of attributes:

- **File** or **global** attributes
- **Dataset** or **data** attributes
- **Dimension** attributes

#### Dimensions

- From the `SDS` class, we can access the dimension names and sizes using the `dimensions()` function.

In [ ]:
ds_dims = sample_ds.dimensions()
ds_dims

While this is nice, the dictionary above does not allow us to access actual dimension objects and the other information that they may hold. To access the objects, we can use the `dim()` function from the `SDS` class.

In [ ]:
sample_dims = list()   

for i in range(len(ds_dims)):
    sample_dims.append(sample_ds.dim(i))
    
sample_dims = tuple(sample_dims)

sample_dims   

Below we can see that we can access not only the labels and size but also the units, scale data type, and number of attributes. We can even access the attributes similarly to the dataset object.

In [ ]:
for i in range(len(sample_dims)):
    print(f"Dimension {i+1}")
    print("\tInfo:", sample_dims[i].info())
    print("\tLength:", sample_dims[i].length())
    print("\tAttributes: ", sample_dims[i].attributes())
    
    # You can access other dim attrs if they exist by this given struture: dim1.attr_name

** * Note from the data type table that dimensions exclusively may have the value 0 or a `SDC.UNLIMITED` data type. This just means that they can grow dynamically **

### Coordinates

In pyHDF, it is possible that coordinates (known as dimension scales) are actually stored as datasets. Thankfully, the `SDS` class provides the `iscoordvar()` function to determine that.

In [ ]:
print(bool(sample_ds.iscoordvar()))

# If there was a scale, it would be accessible via: dim1.getscale()

We can try to traverse through all the datasets and see if one of them holds coordinates.

In [ ]:
coord_sets = list()   # Will hold datasets suspected to be coordinates

for i in range(len(fid.datasets())):
    ds = fid.select(i)
    if ds.iscoordvar():
        coord_sets.append(ds)

In [ ]:
coord_sets

It is also possible that some coordinate information is stored as a file attribute. If we go back once more to our global attribute dictionary, we can see some keys such as corner and bounding coordinates.

In [ ]:
pprint.pprint(fid.attributes())

We can then traverse through it and extract and attributes that may be related to the coordinates.

In [ ]:
coord_attrs = {}   # Will hold coordinate-related attributes
for key, value in fid.attributes().items():
    if 'coordinate' in key.lower() or 'latlong' in key.lower():
        coord_attrs[key] = value

coord_attrs

Looking at these attributes, it would be much easier to work with our bounding coordinates.

If we now go back to our datasets, we can see that thankfully, they all have the same shape.

In [ ]:
count = 1
for vals in fid.datasets().values():
    print(f"Dataset {count} Shape:", vals[0:2])
    count += 1
    shape = vals[1]

Using the bounding coordinates and the shape, we can artificially create our full coordinates; however, this is under the assumption that the datasets truly align with this artificial system. This means that we can't confirm its accuracy.

The first step would be to assign our boundaries.

In [ ]:
for key in coord_attrs.keys():
    if 'north' in key.lower():
        latN = coord_attrs[key]
    if 'south' in key.lower():
        latS = coord_attrs[key]
    if 'east' in key.lower():
        lonE = coord_attrs[key]
    if 'west' in key.lower():
        lonW = coord_attrs[key]

print("North:", latN)
print("South:", latS)
print("East:", lonE)
print("West:", lonW)

Next, we would create our spacing using the dataset shapes and boundaries.

In [ ]:
# Creating our coords. 'lats' can be substituted for 'y' and 'lons' for 'x'
lat_space = (latN - latS) / shape[0]
lon_space = (lonE - lonW) / shape[1]

print(lat_space, "|", lon_space)

Now, we can finally create our coordinates!

In [ ]:
lats = np.linspace(latS, latN, shape[0])
lons = np.linspace(lonW, lonE, shape[1])

In [ ]:
print('Latitudes:\n', lats)
print('Longitudes:\n', lons)

We'll now close our file reader.

In [ ]:
fid.end()

## <font color="red"> Conversion to XArray DataArrays and Datasets</font>

Now that we've been able to get all of the necessary information to create an Xarray Dataset, we can start!

In [ ]:
def get_fid(sample_file):
    """
        Parameter(s): a file name (str)
        Return Type(s): an SD object (file identifier)
        Function: returns our file reader object
    """
    sample_fid = SD(sample_file, SDC.READ)
    return sample_fid

In [ ]:
def get_fill(sample_ds):
    '''
       Parameter(s): SDS object
       Return Type(s): float, int, or None
       Function: returns fill value of a given dataset object
    '''
    for key, value in sample_ds.attributes().items():
        if key == '_FillValue':
            return value 
    return None

In [ ]:
def get_scale(sample_ds):
    '''
       Parameter(s): SDS object
       Return Type(s): float, int, or None
       Function: returns scale factor of a given dataset object
    '''
    for key, value in sample_ds.attributes().items():
        if key == 'scale_factor':
            return value 
    return 1

In [ ]:
def get_offset(sample_ds):
    '''
        Parameter(s): SDS object
        Return Type(s): float, int, or None
        Function: returns offset value of a given dataset object
    '''
    for key, value in sample_ds.attributes().items():
        if key == 'add_offset':
            return value 
    return 0

In [ ]:
def restore_data(sample_ds):
    '''
       Parameter(s): SDS object
       Return Type(s): NumPy array
       Function: restores data of a given dataset object
    '''
    fill = get_fill(sample_ds)
    scale = get_scale(sample_ds)
    offset = get_offset(sample_ds)
    
    data = sample_ds.get()#.astype('float')
    
    data = np.where(data != fill, data, np.nan)
    data *= scale
    data += offset
    
    return data

In [ ]:
def get_dims(sample_ds):
    '''
       Parameter(s): SDS object
       Return Type(s): Python list of SDim objects
       Function: returns dimension objects of a given dataset object
    '''
    sample_dims = []   # Will actually hold dimension objects
    
    for i in range(len(sample_ds.dimensions())):
        sample_dims.append(sample_ds.dim(i))
        
    return sample_dims

In [ ]:
def get_dim_attrs(dim):
    '''
        Parameter(s): SDim object
        Return Type(s): Python Dict
        Function: returns attributes of a given dimension object
    '''
    attrs = dict()
    
    attrs['Given Name'] = dim.info()[0]
    attrs['dtype'] = dim.info()[2]
    attrs.update(dim.attributes())   # Adds other unknown attributes
    
    return attrs

In [ ]:
def get_dims_attrs(ds):
    dims_attrs = dict()
    dims = get_dims(ds)
    
    for dim in dims:
        if dim.info()[0] == 'YDim':
            dims_attrs['lat'] = get_dim_attrs(dim)
        elif dim.info()[0] == 'XDim':
            dims_attrs['lon'] = get_dim_attrs(dim)
            
    return dims_attrs

In [ ]:

def get_fid_coords(sample_fid):
    '''
       Accessing File-level Coords
       Parameter(s): SD object
       Return Type(s): boolean
       Function: returns false if there are no file-level coords and true if there are
       ** Should be modified to return list of file-level coords if there are any
    '''
    coord_sets = list()

    for i in range(len(sample_fid.datasets())):
        sample_ds = sample_fid.select(i)
        if bool(sample_ds.iscoordvar()):
            coord_sets.append(sample_ds)

    if len(coord_sets) > 0:
        return True
    else:
        return False

In [ ]:
def get_coord_bounds(sample_fid):
    '''
       Parameter(s): SD object
       Return Type(s): Python dict of String keys and numeric items
       Function: returns coord boundaries from file-level attrs to construct coords at dataset level
    '''
    coord_attrs = {}    
    # Gets our coordinate-related attributes
    for key, value in sample_fid.attributes().items():
        if 'coordinate' in key.lower():
            coord_attrs[key] = value 
        
    coord_bounds = {}
    # Gets our coordinate bounds
    for key in coord_attrs.keys():
        if 'north' in key.lower():
            coord_bounds['latN'] = coord_attrs[key]
        if 'south' in key.lower():
            coord_bounds['latS'] = coord_attrs[key]
        if 'east' in key.lower():
            coord_bounds['lonE'] = coord_attrs[key]
        if 'west' in key.lower():
            coord_bounds['lonW'] = coord_attrs[key]    
    
    return coord_bounds

In [ ]:
def get_ds_coords(sample_fid, sample_ds): 
    '''
       Accessing Dataset-level Coords
       Parameter(s): an SD and an SDS object
       Return Type(s): Python dict of String keys and NumPy array items
       Function: returns constructed dataset given the file ID object and a particular dataset
    '''
    sample_bounds = get_coord_bounds(sample_fid)
    
    latN = sample_bounds['latN']
    latS = sample_bounds['latS']
    lonE = sample_bounds['lonE']
    lonW = sample_bounds['lonW']
    
    lat_shape = sample_ds.dimensions()['YDim']
    lon_shape = sample_ds.dimensions()['XDim']
    
    if isinstance(sample_bounds, dict):   # Have to configure dataset coords
        lat_space = (latN - latS) / lat_shape
        lon_space = (lonE - lonW) / lon_shape
        
        lats = np.linspace(latS, latN + lat_space, lat_shape)
        lons = np.linspace(lonW, lonE + lon_space, lon_shape)
        
        sample_coords = {'lats': lats, 'lons': lons}
        return sample_coords
        
    #else:   # Coords already set at file level
        #return sample_bounds


In [ ]:
def read_file(file):
    '''
       Creating our XArray Dataset:
       Parameter(s): file name (str)
       Return Type(s): XArray Dataset
       Function: reads a LANDSAT HDF4 file and returns an XArray dataset
    '''
    fid = get_fid(file)
    
    if get_fid_coords(fid):   # File-level coords exist
        pass
    else:
        fid_coords = False    # File-level coords do not exist   
    
    xr_ds = xr.Dataset()
    
    for name in fid.datasets().keys():
        ds = fid.select(name)
        
        coords_dict = get_ds_coords(fid, ds)
        lats = coords_dict['lats']
        lons = coords_dict['lons']
        
        xr_ds[name] = xr.DataArray(restore_data(ds), coords = [lats, lons], dims = ['lat', 'lon'])
        
        xr_ds[name].attrs = ds.attributes()
        
        dims = get_dims(ds)
        for dim in dims:
            if dim.info()[0] == 'YDim':
                xr_ds[name].lat.attrs = get_dim_attrs(dim)
            elif dim.info()[0] == 'XDim':
                xr_ds[name].lon.attrs = get_dim_attrs(dim)
    
    xr_ds.attrs = fid.attributes()
    
    fid.end()
    return xr_ds

In [ ]:
file1_ds = read_file(file_name)

In [ ]:
file1_ds

## <font color="red">Plotting Our Data</font>

In [ ]:
file1_MB = file1_ds.nbytes / 1000000

file1_MB

In [ ]:
var1 = file1_ds['toa_band6']

A basic `matplotlib` plot.

In [ ]:
var1.plot()

A basic `hvPlot` plot.

In [ ]:
var1.hvplot()   # Doesn't work

An intermediate `hvPlot` plot.

In [ ]:
var1.hvplot.quadmesh('lon', 'lat', xlim = (-172.5, -166.75), ylim = (63, 65.3), geo = True, project = True,
                    rasterize = True, projection = ccrs.PlateCarree(), features = ['borders'], coastline = True)

In [ ]:
file2_ds = read_file(file2)

In [ ]:
file2_ds

In [ ]:
file2_MB = file2_ds.nbytes / 1000000

file2_MB

In [ ]:
var2 = file2_ds['toa_band6']

A basic `matplotlib` plot.

In [ ]:
var2.plot()

A basic `hvPlot` plot.

In [ ]:
var2.hvplot()

An intermediate `hvPlot` plot.

In [ ]:
var1.hvplot.contourf('lon', 'lat', 
                     xlim = (-172.5, -166.75), 
                     ylim = (63, 65.3), 
                     geo = True, levels = 10,
                    cmap = 'plasma', 
                     projection = ccrs.PlateCarree(), 
                     features = ['borders'], coastline = True)